In [1]:
import numpy as np
import pandas as pd
import keras

from keras.layers import LSTM, Input, Dense
from keras.models import Model
from keras.utils import to_categorical

/home/vivekb/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 64        
seq_len = 16           
inp_size = 10           
hidden_state_dim = 784 
lstm_dim = 32          

### Model Architecture

In [3]:
## Define MLP
mlp_inp = Input(batch_shape=(batch_size, hidden_state_dim))
mlp_dense = Dense(hidden_state_dim, activation='relu')(mlp_inp)
mlp_model = Model(mlp_inp, mlp_dense)

## Define LSTM model
lstm_inp = Input(batch_shape=(batch_size, seq_len, inp_size))
lstm_layer = LSTM(lstm_dim, stateful=True)(lstm_inp)
lstm_out = Dense(10,activation='softmax')(lstm_layer)
lstm_model = Model(lstm_inp, lstm_out)

## Set the output of MLP as initian cell state
lstm_model.layers[1].states[1] = mlp_model.layers[1].output

## Define the final model
model = Model([mlp_inp, lstm_inp], lstm_model.output)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Test the model: Train the LSTM with MNIST 

In [4]:
from keras.datasets import mnist
data = mnist.load_data()

x, y = data[0]
testx, testy = data[1]
print( x.shape,y.shape)

((60000, 28, 28), (60000,))


In [5]:
n_samples = 4096
x_seq = np.random.normal(size=(n_samples, seq_len, inp_size))
print(x_seq.shape)

(4096, 16, 10)


### Print weights of MLP after each epoch

In [6]:
for i in range(5):
    print(mlp_model.layers[1].get_weights()[0][1][1])
    model.fit([x[:n_samples].reshape(-1,784), x_seq], 
              to_categorical(y[:n_samples],10), 
              batch_size=batch_size, 
              epochs=1,
              verbose=0)

-0.0019691065
-0.0019691065
-0.0019691065
-0.0019691065
-0.0019691065


### Accuracy over epochs

In [7]:
model.fit([x[:n_samples].reshape(-1,784), x_seq], 
              to_categorical(y[:n_samples],10), 
              batch_size=batch_size, 
              epochs=10,
              verbose=1)

Epoch 1/10
4096/4096 [==============================] - 1s - loss: 2.2812 - acc: 0.1331     
Epoch 2/10
4096/4096 [==============================] - 1s - loss: 2.2772 - acc: 0.1409     
Epoch 3/10
4096/4096 [==============================] - 1s - loss: 2.2736 - acc: 0.1460     
Epoch 4/10
4096/4096 [==============================] - 1s - loss: 2.2696 - acc: 0.1538     
Epoch 5/10
4096/4096 [==============================] - 1s - loss: 2.2647 - acc: 0.1558     
Epoch 6/10
4096/4096 [==============================] - 1s - loss: 2.2594 - acc: 0.1582     
Epoch 7/10
4096/4096 [==============================] - 1s - loss: 2.2542 - acc: 0.1619     
Epoch 8/10
4096/4096 [==============================] - 1s - loss: 2.2479 - acc: 0.1646     
Epoch 9/10
4096/4096 [==============================] - 1s - loss: 2.2411 - acc: 0.1709     
Epoch 10/10
4096/4096 [==============================] - 1s - loss: 2.2338 - acc: 0.1687     


### In model summary the MLP part is not included

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (64, 16, 10)              0         
_________________________________________________________________
lstm_1 (LSTM)                (64, 32)                  5504      
_________________________________________________________________
dense_2 (Dense)              (64, 10)                  330       
Total params: 5,834
Trainable params: 5,834
Non-trainable params: 0
_________________________________________________________________
